In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('order_brush_order.csv')
df['event_time'] = df['event_time'].astype('datetime64[ns]')
print(df)

               orderid     shopid     userid          event_time
0       31076582227611   93950878   30530270 2019-12-27 00:23:03
1       31118059853484  156423439   46057927 2019-12-27 11:54:20
2       31123355095755  173699291   67341739 2019-12-27 13:22:35
3       31122059872723   63674025  149380322 2019-12-27 13:01:00
4       31117075665123  127249066  149493217 2019-12-27 11:37:55
...                ...        ...        ...                 ...
222745  31245479170194  110868129  193218997 2019-12-28 23:17:59
222746  31230440360374   91639906    3541807 2019-12-28 19:07:20
222747  31191471145838   29391773  135908070 2019-12-28 08:17:52
222748  31198471732255   61556313   27306783 2019-12-28 10:14:31
222749  31164356713822  147941492  127484153 2019-12-28 00:45:56

[222750 rows x 4 columns]


In [3]:
shop_order = dict()

for row in df.values:
    if shop_order.get(row[1]) is None:
        shop_order[row[1]] = []
        
    shop_order[row[1]].append((row[2], row[3]))
    
for k, v in shop_order.items():
    shop_order[k] = np.asarray(sorted(shop_order[k], key=lambda x: x[1]))

In [4]:
suspicious_detections = dict()

for k, v in shop_order.items():
    for order_row_number, order_row in enumerate(v):
        in_one_hour = [order_row[0]]
        limit_time = order_row[1] + timedelta(hours=1)
        for i in range(order_row_number + 1, len(v)):
            if v[i][1] <= limit_time:
                in_one_hour.append(v[i][0])
            else:
                break
                
        if len(in_one_hour) / len(set(in_one_hour)) >= 3:
            if suspicious_detections.get(k) is None:
                suspicious_detections[k] = set()
            
            in_one_hour_counter = Counter(in_one_hour)
            print(k, in_one_hour_counter)
            max_in_one_hour = max(list(in_one_hour_counter.values()))
            for user in sorted([c_k for c_k, c_v in in_one_hour_counter.items() if c_v == max_in_one_hour]):
                suspicious_detections[k].add(user)
            print(suspicious_detections[k])           

1175477 Counter({122277324: 3})
{122277324}
66861410 Counter({213502289: 3})
{213502289}
8715449 Counter({9753706: 3})
{9753706}
58543771 Counter({61893096: 3})
{61893096}
156883302 Counter({188025647: 5, 62035438: 1})
{188025647}
156883302 Counter({76102350: 5, 187119675: 1})
{76102350, 188025647}
1532569 Counter({181408876: 3})
{181408876}
27476241 Counter({174145893: 5})
{174145893}
27476241 Counter({174145893: 4})
{174145893}
27476241 Counter({174145893: 3})
{174145893}
58835561 Counter({152352709: 3})
{152352709}
80049863 Counter({123158564: 4})
{123158564}
80049863 Counter({123158564: 3})
{123158564}
210197928 Counter({52867898: 4})
{52867898}
210197928 Counter({52867898: 3})
{52867898}
104590058 Counter({81928284: 3})
{81928284}
188546697 Counter({31916119: 4})
{31916119}
188546697 Counter({31916119: 3})
{31916119}
29979455 Counter({107641182: 3})
{107641182}
43412276 Counter({50672876: 3})
{50672876}
68862371 Counter({67554410: 3})
{67554410}
27121667 Counter({183926374: 3})
{1

In [5]:
len(suspicious_detections)

276

In [6]:
result = []

for shopid in shop_order.keys():
    suspicious_users = suspicious_detections.get(shopid, '0')
    if suspicious_users != '0':
        if len(suspicious_users) > 1:
            suspicious_users = '&'.join([str(user) for user in sorted(list(suspicious_users))])
            print(suspicious_users)
        else:
            suspicious_users = list(suspicious_users)[0]
    result.append([shopid, suspicious_users])

76102350&188025647
29857724&212200633
13135622&137245836&162508227&215382704
156202149&214265994
99517130&186080843
1974334&107414154
178491887&191211430
101832161&214208720
187697407&215009429


In [7]:
result_df = pd.DataFrame(result, columns=['shopid', 'userid'])
result_df

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0
...,...,...
18765,163337224,0
18766,772599,0
18767,50236030,0
18768,203587596,0


In [8]:
result_df.to_csv('sub1.csv', index=False)